In [35]:
#Install packages

%pip install faiss-cpu
%pip install sentence-transformers 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [2]:
df = pd.read_csv('sample_text.csv')
df.shape

(8, 2)

In [3]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


<h3>Step 1: Create source embeddings for the text column</h3>

In [4]:
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer('all-mpnet-base-v2')
vectors = encoder.encode(df.text)
vectors.shape

/Users/poojapatel/Desktop/pooja_patel/work/codes/llm/news-research-tool-llm/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(8, 768)

In [5]:
dim = vectors.shape[1]
dim

768

In [6]:
vectors

array([[-0.0024739 ,  0.03626722, -0.05290461, ..., -0.0915236 ,
        -0.03970003, -0.04330488],
       [-0.03357266,  0.00980518, -0.03250131, ..., -0.05165475,
         0.02245891, -0.03156183],
       [-0.01865325, -0.04051302, -0.01235392, ...,  0.00610584,
        -0.07179646,  0.02773845],
       ...,
       [-0.0006646 ,  0.04252126, -0.05645508, ...,  0.01315469,
        -0.03183556, -0.04357661],
       [-0.03317147,  0.03252461, -0.02484839, ...,  0.01174418,
         0.05747125,  0.00571025],
       [-0.00166396,  0.00413834, -0.0459708 , ...,  0.02008529,
         0.0565625 , -0.00161594]], dtype=float32)

<h3>Step 2: Build FAISS index for vectors</h3>

In [7]:
import faiss

index = faiss.IndexFlatL2(dim)
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x288b74c00> >

<h3>Step 3: Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index</h3>

In [8]:
index.add(vectors)

In [9]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x288b74c00> >

<h3>Step 4: Encode search text using same encoder and normalize the output vector</h3>

In [10]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"

vec = encoder.encode(search_query)
vec.shape

(768,)

In [11]:
vec

array([ 1.03882970e-02,  2.78685261e-02, -1.18618011e-02,  1.81326624e-02,
        1.21981266e-03, -1.42995799e-02,  1.96229368e-02,  2.07197666e-02,
       -2.23660581e-02,  4.75626737e-02,  1.77976247e-02, -8.00334942e-03,
        2.53419541e-02,  5.26148938e-02,  8.44944827e-03, -1.63943246e-02,
        1.02661196e-02, -2.74856295e-02,  8.37066919e-02, -1.52885104e-02,
        1.67682618e-02, -3.97137925e-03, -2.74348874e-02,  5.02091721e-02,
       -8.36522598e-03, -4.74074818e-02,  2.36915816e-02, -1.01185357e-02,
       -2.82840673e-02,  7.94266537e-03,  4.21435721e-02, -4.19377722e-03,
       -1.91232767e-02, -3.12784947e-02,  1.24685084e-06, -1.04428250e-02,
       -2.19971612e-02, -8.66928175e-02, -1.88502145e-03, -2.54771374e-02,
       -9.72967036e-03,  7.93310553e-02, -3.55963409e-02, -3.05318594e-04,
       -1.12392679e-02, -3.88931669e-02,  5.49314320e-02,  1.35208055e-01,
       -8.19147453e-02,  1.18784495e-02, -9.01845749e-03,  1.92543976e-02,
       -2.82724053e-02, -

In [12]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec

array([[ 1.03882970e-02,  2.78685261e-02, -1.18618011e-02,
         1.81326624e-02,  1.21981266e-03, -1.42995799e-02,
         1.96229368e-02,  2.07197666e-02, -2.23660581e-02,
         4.75626737e-02,  1.77976247e-02, -8.00334942e-03,
         2.53419541e-02,  5.26148938e-02,  8.44944827e-03,
        -1.63943246e-02,  1.02661196e-02, -2.74856295e-02,
         8.37066919e-02, -1.52885104e-02,  1.67682618e-02,
        -3.97137925e-03, -2.74348874e-02,  5.02091721e-02,
        -8.36522598e-03, -4.74074818e-02,  2.36915816e-02,
        -1.01185357e-02, -2.82840673e-02,  7.94266537e-03,
         4.21435721e-02, -4.19377722e-03, -1.91232767e-02,
        -3.12784947e-02,  1.24685084e-06, -1.04428250e-02,
        -2.19971612e-02, -8.66928175e-02, -1.88502145e-03,
        -2.54771374e-02, -9.72967036e-03,  7.93310553e-02,
        -3.55963409e-02, -3.05318594e-04, -1.12392679e-02,
        -3.88931669e-02,  5.49314320e-02,  1.35208055e-01,
        -8.19147453e-02,  1.18784495e-02, -9.01845749e-0

<h3>Step 5: Search for similar vector in the FAISS index created</h3>

In [13]:
distances, i = index.search(svec, k=2)
i

array([[3, 2]])

In [14]:
df.loc[i[0]]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [16]:
search_query

'I want to buy a polo t-shirt'